In [1]:
from utils import *
debug = True

In [2]:
all_events = [importpbdatapandas(i) for i in range(0, 22948)]

In [3]:
def background_distribution(event1, event2):
    philist = np.array([])
    etalist = np.array([])
    for i in range(len(event1)):
        #hold index0 for first event, and traverse second event
        for j in range(len(event2)):
            dphi = event1["phi"][i] - event2["phi"][j]
            if dphi > np.pi:
                dphi = dphi - 2*np.pi
            if dphi < -np.pi:
                dphi = dphi + 2*np.pi
            deta = event1["eta"][i] - event2["eta"][j]
            philist = np.append(philist, dphi)
            etalist = np.append(etalist, deta)

    df = pd.DataFrame(np.vstack([philist,etalist]).T,columns=['phi','eta'])
    df = pd.concat([df, -1*df], ignore_index=True)

    return df

In [4]:
def background_range(start,end): #简单测试用的: 连续的 different event difference
    # 01,12,23,34....
    if start > end or start < 0 or start > 22947 or end < 0 or end > 22947:
        print("index error")
        return
    
    df = background_distribution(all_events[start], all_events[start+1])
    start += 1
    while start+1 <= end:
        event1 = all_events[start]
        event2 = all_events[start+1]
        temp = background_distribution(event1, event2)
        df = pd.concat([df, temp], ignore_index=True)
        start += 1

    return df
    

In [16]:
def background_range_pro(start, end): #老师让我们去搞的 ： 任意的different event difference
    #01,02,03,04, 12,13,14, 23,34
    t0 = time.time()
    if start > end or start < 0 or start > 22947 or end < 0 or end > 22947:
        print("index error")
        return
    
    current1idx = 0
    current2idx = 1
    # print("now processing index: ",current1idx, current2idx) 
    df = background_distribution(all_events[start], all_events[start+1])
    current2idx += 1
    while current1idx <= end-1:
        print("now processing index A at: ",current1idx, "time: ", time.time()-t0)
        while current2idx <= end:
            # print("now processing index: ",current1idx, current2idx)
            event1 = all_events[current1idx]
            event2 = all_events[current2idx]
            temp = background_distribution(event1, event2)
            df = pd.concat([df, temp], ignore_index=True)
            current2idx += 1
        current1idx += 1
        current2idx = current1idx + 1

    return df


In [5]:
t0 = time.time()

for i in range (367,370): #459: 0-50, 50-100, 100-150, 150-200, 200-250 || 250-300, 300-350, 350-400, 400-459
    idxstart = i*50
    idxend = idxstart + 49 if (idxstart < 22899) else 22947
    #print(idxstart, idxend)

    mixed_i = background_range(idxstart, idxend)
    print("calculating surface for first ", i, "x50 events at time:" , time.time() - t0)
    mixed_x, mixed_y, mixed_z = surfacedata(mixed_i, 40, 40, 3.15, 6)
    filename = 'mixed40x40/mixed_z_' + str(i) + '.csv'
    np.savetxt(filename, mixed_z, delimiter=",")
    # if i == 0:
    #     np.savetxt('mixed40x40/mixed_x_header.csv', mixed_x, delimiter=",")
    #     np.savetxt('mixed40x40/mixed_y_header.csv', mixed_y, delimiter=",")

calculating surface for first  367 x50 events at time: 191.74904799461365
calculating surface for first  368 x50 events at time: 387.09472823143005
calculating surface for first  369 x50 events at time: 564.3462481498718
